A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [21]:
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from mingpt.utils import set_seed

set_seed(3407)

In [22]:
import random

def mult_digit_by_digit(val_a, b_digits):
    res = []
    mul = 1
    for digit in reversed(b_digits):
        res.append(val_a * digit * mul)
        mul *= 10

    res.append(sum(res))
    return res

def pad_number(a, need_len):
    str_a = str(a)
    pad = need_len - len(str_a)
    str_a = pad * '0' + str_a
    return str_a

def random_mult_instance(length):
    a = [random.randint(0,9) for i in range(length)]
    b = [random.randint(0,9) for i in range(length)]

    val_a = int(''.join(str(d) for d in a))

    comp = mult_digit_by_digit(val_a, b)
    need_len = 2*length

    comp = [pad_number(x, need_len) for x in comp]
    return a + b + [int(d) for s in comp for d in s]

random_mult_instance(2)


[0, 6, 2, 5, 0, 0, 3, 0, 0, 1, 2, 0, 0, 1, 5, 0]

In [23]:
class MultDataset(Dataset):
    """ 
    Dataset for the Add problem. E.g. for problem length 3:
    12 + 333 = 345
    Input: 0 1 2 3 3 3 -> Output: 0 3 4 5
    Which will feed into the transformer concatenated as:
    input:  0 1 2 3 3 3 0 3 4
    output: I I I I I 0 3 4 5
    where I is "ignore", as the transformer is reading the input sequence
    """

    def __init__(self, split, length=3):
        assert split in {'train', 'test'}
        self.split = split
        self.length = length
    
    def __len__(self):
        return 10000 # ...
    
    def get_vocab_size(self):
        return 10
    
    def get_block_size(self):
        return 2*self.length + (self.length + 1) * (2 * self.length)

    def __getitem__(self, idx):
        while True:
            rai = random_mult_instance(self.length)
            h = hash(str(rai[:2*self.length]))
            
            inp_split = 'test' if h % 4 == 0 else 'train' # designate 25% of examples as test
            if inp_split == self.split:
                break # ok
        
        x = torch.tensor(rai[:-1], dtype=torch.long)
        y = torch.tensor(rai[1:], dtype=torch.long)
        
        # we only want to predict at output locations, mask out the loss at the input locations
        y[:2*self.length-1] = -1
        return x, y

In [24]:
# print an example instance of the dataset
train_dataset = MultDataset('train')
test_dataset = MultDataset('test')
x, y = train_dataset[0]

print (x)
for a, b in zip(x,y):
    print(int(a),int(b))

tensor([3, 6, 4, 9, 5, 3, 0, 0, 1, 0, 9, 2, 0, 1, 8, 2, 0, 0, 3, 2, 7, 6, 0, 0,
        3, 4, 6, 8, 9])
3 -1
6 -1
4 -1
9 -1
5 -1
3 0
0 0
0 1
1 0
0 9
9 2
2 0
0 1
1 8
8 2
2 0
0 0
0 3
3 2
2 7
7 6
6 0
0 0
0 3
3 4
4 6
6 8
8 9
9 2


In [25]:
# create a GPT instance
from mingpt.model import GPT

model_config = GPT.get_default_config()
model_config.model_type = 'gpt-micro'
#model_config.model_type = 'gpt-nano'

model_config.vocab_size = train_dataset.get_vocab_size()
model_config.block_size = train_dataset.get_block_size()
model = GPT(model_config)

number of parameters: 0.80M


In [26]:
print (model_config.n_head, model_config.n_layer, model_config.n_embd)

4 4 128


In [27]:
# create a Trainer object
from mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-4 # the model we're using is so small that we can go a bit faster
train_config.max_iters = 30000
train_config.num_workers = 0
trainer = Trainer(train_config, model, train_dataset)

running on device cpu


In [28]:
def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")
trainer.set_callback('on_batch_end', batch_end_callback)

trainer.run()

/Users/janek/dev/uwr/language-models/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


iter_dt 0.00ms; iter 0: train loss 2.36803
iter_dt 87.19ms; iter 100: train loss 1.28014
iter_dt 115.63ms; iter 200: train loss 1.08944
iter_dt 118.49ms; iter 300: train loss 0.91356
iter_dt 116.62ms; iter 400: train loss 0.76703
iter_dt 119.54ms; iter 500: train loss 0.60055
iter_dt 115.96ms; iter 600: train loss 0.56211
iter_dt 112.88ms; iter 700: train loss 0.48590
iter_dt 112.57ms; iter 800: train loss 0.43079
iter_dt 105.40ms; iter 900: train loss 0.43017
iter_dt 104.44ms; iter 1000: train loss 0.39121
iter_dt 111.30ms; iter 1100: train loss 0.32052
iter_dt 116.56ms; iter 1200: train loss 0.28754
iter_dt 104.58ms; iter 1300: train loss 0.25755
iter_dt 116.30ms; iter 1400: train loss 0.22159
iter_dt 110.39ms; iter 1500: train loss 0.21877
iter_dt 111.02ms; iter 1600: train loss 0.18469
iter_dt 110.03ms; iter 1700: train loss 0.19993
iter_dt 114.95ms; iter 1800: train loss 0.17058
iter_dt 108.41ms; iter 1900: train loss 0.14142
iter_dt 108.85ms; iter 2000: train loss 0.15757
iter_dt

KeyboardInterrupt: 

In [29]:
# now let's perform some evaluation
model.eval()
None

In [30]:
def eval_add_split(trainer, split, max_batches):
    dataset = {'train':train_dataset, 'test':test_dataset}[split]
    n = train_dataset.length # naugy direct access shrug
    results = []
    mistakes_printed_already = 0
    loader = DataLoader(dataset, batch_size=100, num_workers=0, drop_last=False)

    #loader = DataLoader(dataset, batch_size=1, num_workers=0, drop_last=False)
    for b, (x, y) in enumerate(loader):
        x = x.to(trainer.device)
        y = y.to(trainer.device)

        inp = x[:, :2*n]
        sol = y[:, -2*n:]
        
        cat = model.generate(inp, (n+1)*2*n, do_sample=False) # using greedy argmax, not sampling
        sol_candidate = cat[:, -2*n:]
        correct = (sol == sol_candidate).all(1).cpu() 
        for i in range(x.size(0)):
            results.append(int(correct[i]))
    
    rt = torch.tensor(results, dtype=torch.float)
    print("%s final score: %d/%d = %.2f%% correct" % (split, rt.sum(), len(results), 100*rt.mean()))
    return rt.sum()

# run a lot of examples from both train and test through the model and verify the output correctness
with torch.no_grad():
    train_score = eval_add_split(trainer, 'train', max_batches=50)
    test_score  = eval_add_split(trainer, 'test',  max_batches=50)

train final score: 9739/10000 = 97.39% correct
test final score: 9744/10000 = 97.44% correct
